In this appendix, I will go deeper into the NumPy library for array computing. This
will include more internal detail about the ndarray type and more advanced array
manipulations and algorithms.
  
     
This appendix contains miscellaneous topics and does not necessarily need to be read
linearly.


## A.1 ndarray Object Internals

The NumPy ndarray provides a means to interpret a block of homogeneous data
(either contiguous or strided) as a multidimensional array object. The data type, or
dtype, determines how the data is interpreted as being floating point, integer, boolean,
or any of the other types we’ve been looking at.

Part of what makes ndarray flexible is that every array object is a strided view on a
block of data. You might wonder, for example, how the array view arr[::2, ::-1]
does not copy any data. The reason is that the ndarray is more than just a chunk of
memory and a dtype; it also has “striding” information that enables the array to move
through memory with varying step sizes. More precisely, the ndarray internally con‐
sists of the following:


• A pointer to data—that is, a block of data in RAM or in a memory-mapped file
  
    
• The data type or dtype, describing fixed-size value cells in the array
   
    
• A tuple indicating the array’s shape
   
    
• A tuple of strides, integers indicating the number of bytes to “step” in order to
advance one element along a dimension


For example, a 10 × 5 array would have shape (10, 5):

In [1]:
import numpy as np
np.ones((10, 5)).shape

(10, 5)

A typical (C order) 3 × 4 × 5 array of float64 (8-byte) values has strides (160, 40,
8) (knowing about the strides can be useful because, in general, the larger the strides
on a particular axis, the more costly it is to perform computation along that axis):

In [2]:
np.ones((3, 4, 5), dtype=np.float64).strides

(160, 40, 8)

While it is rare that a typical NumPy user would be interested in the array strides,
they are the critical ingredient in constructing “zero-copy” array views. Strides can
even be negative, which enables an array to move “backward” through memory (this
would be the case, for example, in a slice like obj[::-1] or obj[:, ::-1])

## NumPy dtype Hierarchy

You may occasionally have code that needs to check whether an array contains inte‐
gers, floating-point numbers, strings, or Python objects. Because there are multiple
types of floating-point numbers (float16 through float128), checking that the dtype
is among a list of types would be very verbose. Fortunately, the dtypes have super‐
classes such as np.integer and np.floating, which can be used in conjunction with
the np.issubdtype function:


In [3]:
ints = np.ones(10, dtype=np.uint16)
floats = np.ones(10, dtype=np.float32)

In [4]:
np.issubdtype(ints.dtype, np.integer)

True

In [5]:
np.issubdtype(floats.dtype, np.floating)

True

You can see all of the parent classes of a specific dtype by calling the type’s mro
method:

In [6]:
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

Therefore, we also have:

In [7]:
np.issubdtype(ints.dtype, np.number)

True

Most NumPy users will never have to know about this, but it occasionally comes in
handy. See Figure A-2 for a graph of the dtype hierarchy and parent–subclass
relationships

Some of the dtypes have trailing underscores in their names. These are there to avoid variable name conflicts
between the NumPy-specific types and the Python built-in ones.

## A.2 Advanced Array Manipulation

There are many ways to work with arrays beyond fancy indexing, slicing, and boolean
subsetting. While much of the heavy lifting for data analysis applications is handled
by higher-level functions in pandas, you may at some point need to write a data algo‐
rithm that is not found in one of the existing libraries.

### Reshaping Arrays

In many cases, you can convert an array from one shape to another without copying
any data. To do this, pass a tuple indicating the new shape to the reshape array
instance method. For example, suppose we had a one-dimensional array of values
that we wished to rearrange into a matrix (the result is shown in Figure A-3):


In [8]:
arr = np.arange(8)
arr

array([0, 1, 2, 3, 4, 5, 6, 7])

In [9]:
arr.reshape((4, 2))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

A multidimensional array can also be reshaped:

In [10]:
arr.reshape((4, 2)).reshape((2, 4))

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

One of the passed shape dimensions can be –1, in which case the value used for that
dimension will be inferred from the data:

In [11]:
arr = np.arange(15)
arr.reshape((5, -1))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

Since an array’s shape attribute is a tuple, it can be passed to reshape, too:

In [12]:
other_arr = np.ones((3, 5))
other_arr.shape

(3, 5)

In [13]:
arr.reshape(other_arr.shape)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

The opposite operation of reshape from one-dimensional to a higher dimension is
typically known as flattening or raveling:

In [14]:
arr = np.arange(15).reshape((5, 3))
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [15]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

ravel does not produce a copy of the underlying values if the values in the result
were contiguous in the original array. The flatten method behaves like ravel except
it always returns a copy of the data:

In [16]:
arr.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

The data can be reshaped or raveled in different orders. This is a slightly nuanced
topic for new NumPy users and is therefore the next subtopic.


## C Versus Fortran Order

NumPy gives you control and flexibility over the layout of your data in memory. By
default, NumPy arrays are created in row major order. Spatially this means that if you
have a two-dimensional array of data, the items in each row of the array are stored in
adjacent memory locations. The alternative to row major ordering is column major
order, which means that values within each column of data are stored in adjacent
memory locations.
   
     
For historical reasons, row and column major order are also know as C and Fortran
order, respectively. In the FORTRAN 77 language, matrices are all column major.
   
    
Functions like reshape and ravel accept an order argument indicating the order to
use the data in the array. This is usually set to 'C' or 'F' in most cases (there are also
less commonly used options 'A' and 'K'; see the NumPy documentation, and refer
back to Figure A-3 for an illustration of these options):

In [17]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [18]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [19]:
arr.ravel('F')

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

Reshaping arrays with more than two dimensions can be a bit mind-bending (see
Figure A-3). The key difference between C and Fortran order is the way in which the
dimensions are walked:
   
    
C/row major order
   
    
Traverse higher dimensions first (e.g., axis 1 before advancing on axis 0).
    
     
Fortran/column major order   
    
    
Traverse higher dimensions last (e.g., axis 0 before advancing on axis 1).

## Concatenating and Splitting Arrays

numpy.concatenate takes a sequence (tuple, list, etc.) of arrays and joins them
together in order along the input axis:


In [20]:
arr1 = np.array([[1, 2, 3], [4, 5, 6]])
arr2 = np.array([[7, 8, 9], [10, 11, 12]])

In [21]:
np.concatenate([arr1, arr2], axis=0)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [22]:
np.concatenate([arr1, arr2], axis=1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

There are some convenience functions, like vstack and hstack, for common kinds of
concatenation. The preceding operations could have been expressed as:

In [23]:
np.vstack((arr1, arr2))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [24]:
np.hstack((arr1, arr2))

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

split, on the other hand, slices apart an array into multiple arrays along an axis:

In [25]:
arr = np.random.randn(5, 2)
arr

array([[-1.87470225, -2.88284561],
       [ 1.46335885, -1.42197139],
       [-1.67574206, -0.70737276],
       [ 0.14334176, -0.79872908],
       [ 0.41594851,  0.80290547]])

In [26]:
first, second, third = np.split(arr, [1, 3])
first

array([[-1.87470225, -2.88284561]])

In [27]:
second

array([[ 1.46335885, -1.42197139],
       [-1.67574206, -0.70737276]])

In [28]:
third

array([[ 0.14334176, -0.79872908],
       [ 0.41594851,  0.80290547]])

The value [1, 3] passed to np.split indicate the indices at which to split the array
into pieces.

Table A-1. Array concatenation functions
----------------------------------------------------------
    
Function --> Description
  
   
concatenate --> Most general function, concatenates collection of arrays along one axis
  
    
vstack, row_stack --> Stack arrays row-wise (along axis 0)
   
    
hstack --> Stack arrays column-wise (along axis 1)
  
    
column_stack --> Like hstack, but converts 1D arrays to 2D column vectors first
  
    
dstack --> Stack arrays “depth”-wise (along axis 2)
   
    
split --> Split array at passed locations along a particular axis
  
    
hsplit/vsplit --> Convenience functions for splitting on axis 0 and 1, respectively

### Stacking helpers: r_ and c_


There are two special objects in the NumPy namespace, r_ and c_, that make stacking
arrays more concise:


In [29]:
arr = np.arange(6)
arr1 = arr.reshape((3, 2))
arr2 = np.random.randn(3, 2)
np.r_[arr1, arr2]

array([[ 0.        ,  1.        ],
       [ 2.        ,  3.        ],
       [ 4.        ,  5.        ],
       [-0.00928597,  0.47962154],
       [-1.58326604,  1.10585653],
       [-1.50165706, -0.74198821]])

In [30]:
np.c_[np.r_[arr1, arr2], arr]

array([[ 0.        ,  1.        ,  0.        ],
       [ 2.        ,  3.        ,  1.        ],
       [ 4.        ,  5.        ,  2.        ],
       [-0.00928597,  0.47962154,  3.        ],
       [-1.58326604,  1.10585653,  4.        ],
       [-1.50165706, -0.74198821,  5.        ]])

These additionally can translate slices to arrays:

In [31]:
np.c_[1:6, -10:-5]


array([[  1, -10],
       [  2,  -9],
       [  3,  -8],
       [  4,  -7],
       [  5,  -6]])

see the docstring for more on what you can do with c_ and r_.

## Repeating Elements: tile and repeat

Two useful tools for repeating or replicating arrays to produce larger arrays are the
repeat and tile functions. repeat replicates each element in an array some number
of times, producing a larger array:

In [32]:
arr = np.arange(3)
arr

array([0, 1, 2])

In [33]:
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

The need to replicate or repeat arrays can be less common with
NumPy than it is with other array programming frameworks like
MATLAB. One reason for this is that broadcasting often fills this
need better, which is the subject of the next section.

By default, if you pass an integer, each element will be repeated that number of times.
If you pass an array of integers, each element can be repeated a different number of
times:

In [34]:
arr.repeat([2, 3, 4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

Multidimensional arrays can have their elements repeated along a particular axis.

In [35]:
arr = np.random.randn(2, 2)
arr

array([[-0.29978315, -0.35428312],
       [-1.09718178, -1.04212578]])

In [36]:
arr.repeat(2, axis=0)

array([[-0.29978315, -0.35428312],
       [-0.29978315, -0.35428312],
       [-1.09718178, -1.04212578],
       [-1.09718178, -1.04212578]])

Note that if no axis is passed, the array will be flattened first, which is likely not what
you want. Similarly, you can pass an array of integers when repeating a multidimen‐
sional array to repeat a given slice a different number of times:


In [37]:
arr.repeat([2, 3], axis=0)

array([[-0.29978315, -0.35428312],
       [-0.29978315, -0.35428312],
       [-1.09718178, -1.04212578],
       [-1.09718178, -1.04212578],
       [-1.09718178, -1.04212578]])

In [38]:
arr.repeat([2, 3], axis=1)

array([[-0.29978315, -0.29978315, -0.35428312, -0.35428312, -0.35428312],
       [-1.09718178, -1.09718178, -1.04212578, -1.04212578, -1.04212578]])

tile, on the other hand, is a shortcut for stacking copies of an array along an axis.
Visually you can think of it as being akin to “laying down tiles”:


In [39]:
arr

array([[-0.29978315, -0.35428312],
       [-1.09718178, -1.04212578]])

In [40]:
np.tile(arr, 2)

array([[-0.29978315, -0.35428312, -0.29978315, -0.35428312],
       [-1.09718178, -1.04212578, -1.09718178, -1.04212578]])

The second argument is the number of tiles; with a scalar, the tiling is made row by
row, rather than column by column. The second argument to tile can be a tuple
indicating the layout of the “tiling”:


In [41]:
arr

array([[-0.29978315, -0.35428312],
       [-1.09718178, -1.04212578]])

In [42]:
np.tile(arr, (2, 1))

array([[-0.29978315, -0.35428312],
       [-1.09718178, -1.04212578],
       [-0.29978315, -0.35428312],
       [-1.09718178, -1.04212578]])

In [43]:
np.tile(arr, (3, 2))

array([[-0.29978315, -0.35428312, -0.29978315, -0.35428312],
       [-1.09718178, -1.04212578, -1.09718178, -1.04212578],
       [-0.29978315, -0.35428312, -0.29978315, -0.35428312],
       [-1.09718178, -1.04212578, -1.09718178, -1.04212578],
       [-0.29978315, -0.35428312, -0.29978315, -0.35428312],
       [-1.09718178, -1.04212578, -1.09718178, -1.04212578]])

## Fancy Indexing Equivalents: take and put

As you may recall from Chapter 4, one way to get and set subsets of arrays is by fancy
indexing using integer arrays

In [44]:
arr = np.arange(10) * 100
inds = [7, 1, 2, 6]
arr[inds]

array([700, 100, 200, 600])

There are alternative ndarray methods that are useful in the special case of only mak‐
ing a selection on a single axis:


In [45]:
arr.take(inds)

array([700, 100, 200, 600])

In [46]:
arr.put(inds, 42)

In [47]:
arr

array([  0,  42,  42, 300, 400, 500,  42,  42, 800, 900])

In [48]:
arr.put(inds, [40, 41, 42, 43])

In [49]:
arr


array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

To use take along other axes, you can pass the axis keyword:


In [50]:
inds = [2, 0, 2, 1]
arr = np.random.randn(2, 4)
arr

array([[-0.24988627, -1.46969707,  0.43291802,  0.32978433],
       [ 0.25974421,  0.33712148,  0.56181722,  0.19406534]])

In [51]:
arr.take(inds, axis=1)

array([[ 0.43291802, -0.24988627,  0.43291802, -1.46969707],
       [ 0.56181722,  0.25974421,  0.56181722,  0.33712148]])

put does not accept an axis argument but rather indexes into the flattened (onedimensional, C order) version of the array. Thus, when you need to set elements
using an index array on other axes, it is often easiest to use fancy indexing.

## A.3 Broadcasting

Broadcasting describes how arithmetic works between arrays of different shapes. It
can be a powerful feature, but one that can cause confusion, even for experienced
users. The simplest example of broadcasting occurs when combining a scalar value
with an array

In [52]:
arr = np.arange(5)
arr

array([0, 1, 2, 3, 4])

In [53]:
arr * 4

array([ 0,  4,  8, 12, 16])

Here we say that the scalar value 4 has been broadcast to all of the other elements in
the multiplication operation.  
    
For example, we can demean each column of an array by subtracting the column
means. In this case, it is very simple:

In [54]:
arr = np.random.randn(4, 3)
arr.mean(0)

array([-0.09645966, -0.97708809, -1.20681838])

In [55]:
demeaned = arr - arr.mean(0)
demeaned

array([[ 0.75186523, -0.33558519, -0.55125226],
       [-0.99273085,  0.32784655, -0.25875524],
       [ 0.52781434,  0.25545625,  1.41378975],
       [-0.28694872, -0.24771761, -0.60378225]])

In [56]:
demeaned.mean(0)

array([ 0.00000000e+00,  0.00000000e+00, -1.11022302e-16])

See Figure A-4 for an illustration of this operation. Demeaning the rows as a broad‐
cast operation requires a bit more care. Fortunately, broadcasting potentially lower
dimensional values across any dimension of an array (like subtracting the row means
from each column of a two-dimensional array) is possible as long as you follow the
rules.


The Broadcasting Rule
   
      
Two arrays are compatible for broadcasting if for each trailing dimension (i.e., starting
from the end) the axis lengths match or if either of the lengths is 1. Broadcasting is
then performed over the missing or length 1 dimensions.


Even as an experienced NumPy user, I often find myself having to pause and draw a
diagram as I think about the broadcasting rule. Consider the last example and sup‐
pose we wished instead to subtract the mean value from each row. Since arr.mean(0)
has length 3, it is compatible for broadcasting across axis 0 because the trailing
dimension in arr is 3 and therefore matches. According to the rules, to subtract over
axis 1 (i.e., subtract the row mean from each row), the smaller array must have shape
(4, 1):

In [57]:
arr


array([[ 0.65540558, -1.31267328, -1.75807064],
       [-1.08919051, -0.64924154, -1.46557362],
       [ 0.43135468, -0.72163184,  0.20697137],
       [-0.38340837, -1.22480571, -1.81060063]])

In [58]:
row_means = arr.mean(1)
row_means.shape

(4,)

In [59]:
row_means.reshape((4, 1))

array([[-0.80511278],
       [-1.06800189],
       [-0.0277686 ],
       [-1.1396049 ]])

In [60]:
demeaned = arr - row_means.reshape((4, 1))
demeaned.mean(1)

array([-7.40148683e-17,  3.70074342e-17,  9.25185854e-18,  0.00000000e+00])

### Broadcasting Over Other Axes
  
Broadcasting with higher dimensional arrays can seem even more mind-bending, but
it is really a matter of following the rules. If you don’t, you’ll get an error like this:     

In [61]:
arr - arr.mean(1)

ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

It’s quite common to want to perform an arithmetic operation with a lower dimen‐
sional array across axes other than axis 0. According to the broadcasting rule, the
“broadcast dimensions” must be 1 in the smaller array. In the example of row
demeaning shown here, this meant reshaping the row means to be shape (4, 1)
instead of (4,)

In [ ]:
arr - arr.mean(1).reshape((4, 1))

In the three-dimensional case, broadcasting over any of the three dimensions is only
a matter of reshaping the data to be shape-compatible. Figure A-7 nicely visualizes the
shapes required to broadcast over each axis of a three-dimensional array

A common problem, therefore, is needing to add a new axis with length 1 specifically
for broadcasting purposes. Using reshape is one option, but inserting an axis
requires constructing a tuple indicating the new shape. This can often be a tedious
exercise. Thus, NumPy arrays offer a special syntax for inserting new axes by index‐
ing. We use the special np.newaxis attribute along with “full” slices to insert the new
axis:

In [ ]:
arr = np.zeros((4, 4))
arr_3d = arr[:, np.newaxis, :]
arr_3d.shape

In [ ]:
arr_1d = np.random.normal(size=3)
arr_1d[:, np.newaxis]

In [ ]:
arr_1d[np.newaxis, :]

Thus, if we had a three-dimensional array and wanted to demean axis 2, say, we
would need to write:


In [ ]:
arr = np.random.randn(3, 4, 5)
depth_means = arr.mean(2)
depth_means

In [ ]:
depth_means.shape


In [ ]:
demeaned = arr - depth_means[:, :, np.newaxis]
demeaned.mean(2)

You might be wondering if there’s a way to generalize demeaning over an axis without
sacrificing performance. There is, but it requires some indexing gymnastics:


In [ ]:
def demean_axis(arr, axis=0):
    means = arr.mean(axis)
    # This generalizes things like [:, :, np.newaxis] to N dimensions
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis
    return arr - means[indexer]

## Setting Array Values by Broadcasting

The same broadcasting rule governing arithmetic operations also applies to setting
values via array indexing. In a simple case, we can do things like:

In [ ]:
arr = np.zeros((4, 3))
arr[:] = 5
arr

However, if we had a one-dimensional array of values we wanted to set into the col‐
umns of the array, we can do that as long as the shape is compatible:

In [ ]:
col = np.array([1.28, -0.42, 0.44, 1.6])
arr[:] = col[:, np.newaxis]
arr

In [ ]:
arr[:2] = [[-1.37], [0.509]]
arr

## A.4 Advanced ufunc Usage

While many NumPy users will only make use of the fast element-wise operations pro‐
vided by the universal functions, there are a number of additional features that occa‐
sionally can help you write more concise code without loops.

### ufunc Instance Methods

Each of NumPy’s binary ufuncs has special methods for performing certain kinds of
special vectorized operations. These are summarized in Table A-2, but I’ll give a few
concrete examples to illustrate how they work.

reduce takes a single array and aggregates its values, optionally along an axis, by per‐
forming a sequence of binary operations. For example, an alternative way to sum ele‐
ments in an array is to use np.add.reduce:


In [ ]:
arr = np.arange(10)
np.add.reduce(arr)

In [ ]:
arr.sum()

The starting value (0 for add) depends on the ufunc. If an axis is passed, the reduction
is performed along that axis. This allows you to answer certain kinds of questions in a
concise way. As a less trivial example, we can use np.logical_and to check whether
the values in each row of an array are sorted:


In [ ]:
np.random.seed(12346) # for reproducibility
arr = np.random.randn(5, 5)
arr[::2].sort(1) # sort a few rows
arr[:, :-1] < arr[:, 1:]

In [ ]:
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:], axis=1)

Note that logical_and.reduce is equivalent to the all method.


accumulate is related to reduce like cumsum is related to sum. It produces an array of
the same size with the intermediate “accumulated” values:

In [ ]:
arr = np.arange(15).reshape((3, 5))
np.add.accumulate(arr, axis=1)

outer performs a pairwise cross-product between two arrays:

In [ ]:
arr = np.arange(3).repeat([1, 2, 2])
arr

In [ ]:
np.multiply.outer(arr, np.arange(5))

The output of outer will have a dimension that is the sum of the dimensions of the
inputs:


In [ ]:
x, y = np.random.randn(3, 4), np.random.randn(5)
result = np.subtract.outer(x, y)

In [ ]:
 result.shape

The last method, reduceat, performs a “local reduce,” in essence an array groupby
operation in which slices of the array are aggregated together. It accepts a sequence of
“bin edges” that indicate how to split and aggregate the values:


In [ ]:
arr = np.arange(10)
np.add.reduceat(arr, [0, 5, 8])

The results are the reductions (here, sums) performed over arr[0:5], arr[5:8], and
arr[8:]. As with the other methods, you can pass an axis argument:

In [ ]:
arr = np.multiply.outer(np.arange(4), np.arange(5))
arr

In [ ]:
np.add.reduceat(arr, [0, 2, 4], axis=1)

Table A-2. ufunc methods
-------------------------------------
   
   
Method --> Description
   
    
reduce(x) --> Aggregate values by successive applications of the operation
   
    
accumulate(x) --> Aggregate values, preserving all partial aggregates
    
     
reduceat(x, bins) --> “Local” reduce or “group by”; reduce contiguous slices of data to produce aggregated array
   
     
outer(x, y) --> Apply operation to all pairs of elements in x and y; the resulting array has shape x.shape +
y.shape

## Writing New ufuncs in Python

There are a number of facilities for creating your own NumPy ufuncs. The most gen‐
eral is to use the NumPy C API, but that is beyond the scope of this book. In this
section, we will look at pure Python ufuncs.
   
     
numpy.frompyfunc accepts a Python function along with a specification for the num‐
ber of inputs and outputs. For example, a simple function that adds element-wise
would be specified as:

In [ ]:
def add_elements(x, y):
    return x + y
add_them = np.frompyfunc(add_elements, 2, 1)
add_them(np.arange(8), np.arange(8))

Functions created using frompyfunc always return arrays of Python objects, which
can be inconvenient. Fortunately, there is an alternative (but slightly less featureful)
function, numpy.vectorize, that allows you to specify the output type:


In [ ]:
add_them = np.vectorize(add_elements, otypes=[np.float64])
add_them(np.arange(8), np.arange(8))

These functions provide a way to create ufunc-like functions, but they are very slow
because they require a Python function call to compute each element, which is a lot
slower than NumPy’s C-based ufunc loops:


In [ ]:
arr = np.random.randn(10000)
%timeit add_them(arr, arr)

In [ ]:
%timeit np.add(arr, arr)


Later in this chapter we’ll show how to create fast ufuncs in Python using the Numba
project.

## A.5 Structured and Record Arrays

You may have noticed up until now that ndarray is a homogeneous data container;
that is, it represents a block of memory in which each element takes up the same
number of bytes, determined by the dtype. On the surface, this would appear to not
allow you to represent heterogeneous or tabular-like data. A structured array is an
ndarray in which each element can be thought of as representing a struct in C (hence
the “structured” name) or a row in a SQL table with multiple named fields:

In [ ]:
dtype = [('x', np.float64), ('y', np.int32)]
sarr = np.array([(1.5, 6), (np.pi, -2)], dtype=dtype)
sarr

There are several ways to specify a structured dtype (see the online NumPy documen‐
tation). One typical way is as a list of tuples with (field_name, field_data_type).
Now, the elements of the array are tuple-like objects whose elements can be accessed
like a dictionary:

In [ ]:
sarr[0]

In [ ]:
sarr[0]['y']


The field names are stored in the dtype.names attribute. When you access a field on
the structured array, a strided view on the data is returned, thus copying nothing:


In [ ]:
sarr['x']

## Nested dtypes and Multidimensional Fields

When specifying a structured dtype, you can additionally pass a shape (as an int or
tuple):


In [ ]:
dtype = [('x', np.int64, 3), ('y', np.int32)]
arr = np.zeros(4, dtype=dtype)
arr

In this case, the x field now refers to an array of length 3 for each record:

In [ ]:
arr[0]['x']

Conveniently, accessing arr['x'] then returns a two-dimensional array instead of a
one-dimensional array as in prior examples:

In [ ]:
arr['x']

This enables you to express more complicated, nested structures as a single block of
memory in an array. You can also nest dtypes to make more complex structures. Here
is an example:

In [ ]:
dtype = [('x', [('a', 'f8'), ('b', 'f4')]), ('y', np.int32)]
data = np.array([((1, 2), 5), ((3, 4), 6)], dtype=dtype)
data['x']

In [ ]:
data['y']

In [ ]:
data['x']['a']

pandas DataFrame does not support this feature directly, though it is similar to hier‐
archical indexing

## Why Use Structured Arrays?

Compared with, say, a pandas DataFrame, NumPy structured arrays are a compara‐
tively low-level tool. They provide a means to interpreting a block of memory as a
tabular structure with arbitrarily complex nested columns. Since each element in the
array is represented in memory as a fixed number of bytes, structured arrays provide a very fast and efficient way of writing data to and from disk (including memory
maps), transporting it over the network, and other such uses.
   
     
As another common use for structured arrays, writing data files as fixed-length
record byte streams is a common way to serialize data in C and C++ code, which is
commonly found in legacy systems in industry. As long as the format of the file is
known (the size of each record and the order, byte size, and data type of each ele‐
ment), the data can be read into memory with np.fromfile. Specialized uses like this
are beyond the scope of this book, but it’s worth knowing that such things are
possible.


## A.6 More About Sorting


Like Python’s built-in list, the ndarray sort instance method is an in-place sort,
meaning that the array contents are rearranged without producing a new array:

In [62]:
arr = np.random.randn(6)
arr.sort()
arr

array([-1.40479231, -1.00598515, -0.06112872,  0.43083776,  0.89805025,
        1.53620797])

When sorting arrays in-place, remember that if the array is a view on a different
ndarray, the original array will be modified:

In [63]:
arr = np.random.randn(3, 5)
arr

array([[-0.48280318, -1.29146901, -1.36939171,  0.28323385,  0.01749174],
       [-1.02062275, -1.61938026,  2.03126714,  0.27259195,  0.8055873 ],
       [-0.53258276,  1.24332626,  0.66350596,  1.50702249,  0.26298008]])

In [64]:
arr[:, 0].sort() # Sort first column values in-place
arr

array([[-1.02062275, -1.29146901, -1.36939171,  0.28323385,  0.01749174],
       [-0.53258276, -1.61938026,  2.03126714,  0.27259195,  0.8055873 ],
       [-0.48280318,  1.24332626,  0.66350596,  1.50702249,  0.26298008]])

On the other hand, numpy.sort creates a new, sorted copy of an array. Otherwise, it
accepts the same arguments (such as kind) as ndarray.sort:

In [65]:
arr = np.random.randn(5)
arr

array([-0.07658012,  0.7605264 ,  0.75664765, -0.72502629,  1.31647686])

In [66]:
np.sort(arr)

array([-0.72502629, -0.07658012,  0.75664765,  0.7605264 ,  1.31647686])

In [67]:
arr

array([-0.07658012,  0.7605264 ,  0.75664765, -0.72502629,  1.31647686])

All of these sort methods take an axis argument for sorting the sections of data along
the passed axis independently:


In [68]:
arr = np.random.randn(3, 5)
arr

array([[ 1.46070377, -2.18987953, -1.06358666, -2.11025476, -0.50154829],
       [ 0.29493964,  1.31047285, -1.79364174, -1.57961459, -0.27850366],
       [ 1.43159359, -1.2303962 , -1.80271999,  0.1255935 ,  0.33716273]])

In [69]:
arr.sort(axis=1)
arr

array([[-2.18987953, -2.11025476, -1.06358666, -0.50154829,  1.46070377],
       [-1.79364174, -1.57961459, -0.27850366,  0.29493964,  1.31047285],
       [-1.80271999, -1.2303962 ,  0.1255935 ,  0.33716273,  1.43159359]])

You may notice that none of the sort methods have an option to sort in descending
order. This is a problem in practice because array slicing produces views, thus not
producing a copy or requiring any computational work. Many Python users are
familiar with the “trick” that for a list values, values[::-1] returns a list in reverse
order. The same is true for ndarrays:

In [70]:
arr[:, ::-1]

array([[ 1.46070377, -0.50154829, -1.06358666, -2.11025476, -2.18987953],
       [ 1.31047285,  0.29493964, -0.27850366, -1.57961459, -1.79364174],
       [ 1.43159359,  0.33716273,  0.1255935 , -1.2303962 , -1.80271999]])

## Indirect Sorts: argsort and lexsort


In data analysis you may need to reorder datasets by one or more keys. For example, a
table of data about some students might need to be sorted by last name, then by first
name. This is an example of an indirect sort, and if you’ve read the pandas-related
chapters you have already seen many higher-level examples. Given a key or keys (an
array of values or multiple arrays of values), you wish to obtain an array of integer
indices (I refer to them colloquially as indexers) that tells you how to reorder the data
to be in sorted order. Two methods for this are argsort and numpy.lexsort. As an
example:


In [71]:
values = np.array([5, 0, 1, 3, 2])
indexer = values.argsort()

In [72]:
indexer

array([1, 2, 4, 3, 0], dtype=int64)

In [73]:
values[indexer]

array([0, 1, 2, 3, 5])

As a more complicated example, this code reorders a two-dimensional array by its
first row

In [74]:
arr = np.random.randn(3, 5)
arr[0] = values
arr

array([[ 5.        ,  0.        ,  1.        ,  3.        ,  2.        ],
       [ 0.87170034, -1.26754422,  0.57788205,  2.18826431,  0.06390339],
       [ 1.30788948,  0.61541111,  0.39766697,  1.19030574, -0.56683756]])

In [75]:
arr[:, arr[0].argsort()]

array([[ 0.        ,  1.        ,  2.        ,  3.        ,  5.        ],
       [-1.26754422,  0.57788205,  0.06390339,  2.18826431,  0.87170034],
       [ 0.61541111,  0.39766697, -0.56683756,  1.19030574,  1.30788948]])

lexsort is similar to argsort, but it performs an indirect lexicographical sort on multi‐
ple key arrays. Suppose we wanted to sort some data identified by first and last
names:

In [76]:
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])
sorter = np.lexsort((first_name, last_name))
sorter

array([1, 2, 3, 0, 4], dtype=int64)

In [77]:
zip(last_name[sorter], first_name[sorter])

lexsort can be a bit confusing the first time you use it because the order in which the
keys are used to order the data starts with the last array passed. Here, last_name was
used before first_name.

pandas methods like Series’s and DataFrame’s sort_values method
are implemented with variants of these functions (which also must
take into account missing values).

## Alternative Sort Algorithms

A stable sorting algorithm preserves the relative position of equal elements. This can
be especially important in indirect sorts where the relative ordering is meaningful:

In [78]:
values = np.array(['2:first', '2:second', '1:first', '1:second','1:third'])
key = np.array([2, 2, 1, 1, 1])
indexer = key.argsort(kind='mergesort')
indexer

array([2, 3, 4, 0, 1], dtype=int64)

In [79]:
values.take(indexer)

array(['1:first', '1:second', '1:third', '2:first', '2:second'],
      dtype='<U8')

The only stable sort available is mergesort, which has guaranteed O(n log n) perfor‐
mance (for complexity buffs), but its performance is on average worse than the
default quicksort method. See Table A-3 for a summary of available methods and
their relative performance (and performance guarantees). This is not something that
most users will ever have to think about, but it’s useful to know that it’s there.


Table A-3. Array sorting methods
------------------------------------------------

Kind --> Speed --> Stable --> Work space --> Worst case
  
    
'quicksort' --> 1 --> No --> 0 --> O(n^2)
  
    
'mergesort' --> 2 --> Yes --> n / 2 --> O(n log n)
    
    
'heapsort' --> 3 --> No --> 0 --> O(n log n)

## Partially Sorting Arrays

One of the goals of sorting can be to determine the largest or smallest elements in an
array. NumPy has optimized methods, numpy.partition and np.argpartition, for
partitioning an array around the k-th smallest element:

In [80]:
np.random.seed(12345)
arr = np.random.randn(20)
arr

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057,
        1.39340583,  0.09290788,  0.28174615,  0.76902257,  1.24643474,
        1.00718936, -1.29622111,  0.27499163,  0.22891288,  1.35291684,
        0.88642934, -2.00163731, -0.37184254,  1.66902531, -0.43856974])

In [81]:
np.partition(arr, 3)

array([-2.00163731, -1.29622111, -0.5557303 , -0.51943872, -0.37184254,
       -0.43856974, -0.20470766,  0.28174615,  0.76902257,  0.47894334,
        1.00718936,  0.09290788,  0.27499163,  0.22891288,  1.35291684,
        0.88642934,  1.39340583,  1.96578057,  1.66902531,  1.24643474])

After you call partition(arr, 3), the first three elements in the result are the small‐
est three values in no particular order. numpy.argpartition, similar to numpy.arg
sort, returns the indices that rearrange the data into the equivalent order:

In [82]:
indices = np.argpartition(arr, 3)
indices

array([16, 11,  3,  2, 17, 19,  0,  7,  8,  1, 10,  6, 12, 13, 14, 15,  5,
        4, 18,  9], dtype=int64)

In [83]:
arr.take(indices)

array([-2.00163731, -1.29622111, -0.5557303 , -0.51943872, -0.37184254,
       -0.43856974, -0.20470766,  0.28174615,  0.76902257,  0.47894334,
        1.00718936,  0.09290788,  0.27499163,  0.22891288,  1.35291684,
        0.88642934,  1.39340583,  1.96578057,  1.66902531,  1.24643474])

## numpy.searchsorted: Finding Elements in a Sorted Array

searchsorted is an array method that performs a binary search on a sorted array,
returning the location in the array where the value would need to be inserted to
maintain sortedness:


In [84]:
arr = np.array([0, 1, 7, 12, 15])
arr.searchsorted(9)

3

You can also pass an array of values to get an array of indices back:

In [85]:
arr.searchsorted([0, 8, 11, 16])

array([0, 3, 3, 5], dtype=int64)

You might have noticed that searchsorted returned 0 for the 0 element. This is
because the default behavior is to return the index at the left side of a group of equal
values:

In [86]:
arr = np.array([0, 0, 0, 1, 1, 1, 1])
arr.searchsorted([0, 1])

array([0, 3], dtype=int64)

In [87]:
arr.searchsorted([0, 1], side='right')

array([3, 7], dtype=int64)

As another application of searchsorted, suppose we had an array of values between
0 and 10,000, and a separate array of “bucket edges” that we wanted to use to bin the
data:

In [88]:
data = np.floor(np.random.uniform(0, 10000, size=50))
bins = np.array([0, 100, 1000, 5000, 10000])
data

array([9940., 6768., 7908., 1709.,  268., 8003., 9037.,  246., 4917.,
       5262., 5963.,  519., 8950., 7282., 8183., 5002., 8101.,  959.,
       2189., 2587., 4681., 4593., 7095., 1780., 5314., 1677., 7688.,
       9281., 6094., 1501., 4896., 3773., 8486., 9110., 3838., 3154.,
       5683., 1878., 1258., 6875., 7996., 5735., 9732., 6340., 8884.,
       4954., 3516., 7142., 5039., 2256.])

To then get a labeling of which interval each data point belongs to (where 1 would
mean the bucket [0, 100)), we can simply use searchsorted:

In [89]:
labels = bins.searchsorted(data)
labels

array([4, 4, 4, 3, 2, 4, 4, 2, 3, 4, 4, 2, 4, 4, 4, 4, 4, 2, 3, 3, 3, 3,
       4, 3, 4, 3, 4, 4, 4, 3, 3, 3, 4, 4, 3, 3, 4, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 4, 4, 3], dtype=int64)

This, combined with pandas’s groupby, can be used to bin data:

In [91]:
import pandas as pd
pd.Series(data).groupby(labels).mean()

2     498.000000
3    3064.277778
4    7389.035714
dtype: float64

## A.7 Writing Fast NumPy Functions with Numba

Numba is an open source project that creates fast functions for NumPy-like data
using CPUs, GPUs, or other hardware. It uses the LLVM Project to translate Python
code into compiled machine code.


To introduce Numba, let’s consider a pure Python function that computes the expres‐
sion (x - y).mean() using a for loop:

In [92]:
import numpy as np
def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

This function is very slow:


In [93]:
x = np.random.randn(10000000)
y = np.random.randn(10000000)
%timeit mean_distance(x, y)

5.34 s ± 80.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [94]:
x = np.random.randn(10000000)
In [210]: y = np.random.randn(10000000)
In [211]: %timeit mean_distance(x, y)

5.18 s ± 191 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The NumPy version is over 100 times faster. We can turn this function into a com‐
piled Numba function using the numba.jit function:


In [98]:
import numba as nb
numba_mean_distance = nb.jit(mean_distance)

ModuleNotFoundError: No module named 'numba'

In [96]:
! pip install numba

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Practice_Code\Python_Practice\Python_For_Data_Analysis\myenv\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Practice_Code\Python_Practice\Python_For_Data_Analysis\myenv\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Practice_Code\Python_Practice\Python_For_Data_Analysis\myenv\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Practice_Code\Python_Practice\Python_For_Data_Analysis\myenv\lib\http\client.py", line 463, in read




During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Practice_Code\Python_Practice\Python_For_Data_Analysis\myenv\lib\site-packages\pip\_internal\cli\base_command.py", line 173, in _main
    status = self.run(options, args)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Practice_Code\Python_Practice\Python_For_Data_Analysis\myenv\lib\site-packages\pip\_internal\cli\req_command.py", line 203, in wrapper
    return func(self, options, args)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Practice_Code\Python_Practice\Python_For_Data_Analysis\myenv\lib\site-packages\pip\_internal\commands\install.py", line 316, in run
    reqs, check_supported_wheels=not options.target_dir
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Practice_Code\Python_Pr

We could also have written this as a decorator:


In [97]:
@nb.jit
def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

NameError: name 'nb' is not defined

The resulting function is actually faster than the vectorized NumPy version:

In [ ]:
%timeit numba_mean_distance(x, y)

Numba cannot compile arbitrary Python code, but it supports a significant subset of
pure Python that is most useful for writing numerical algorithms.


Numba is a deep library, supporting different kinds of hardware, modes of compila‐
tion, and user extensions. It is also able to compile a substantial subset of the NumPy
Python API without explicit for loops. Numba is able to recognize constructs that
can be compiled to machine code, while substituting calls to the CPython API for
functions that it does not know how to compile. Numba’s jit function has an option,
nopython=True, which restricts allowed code to Python code that can be compiled to
LLVM without any Python C API calls. jit(nopython=True) has a shorter alias
numba.njit.

In the previous example, we could have written:

In [99]:
from numba import float64, njit
@njit(float64(float64[:], float64[:]))
def mean_distance(x, y):
    return (x - y).mean()

ModuleNotFoundError: No module named 'numba'

I encourage you to learn more by reading the online documentation for Numba. The
next section shows an example of creating custom NumPy ufunc objects.


## Creating Custom numpy.ufunc Objects with Numba

The numba.vectorize function creates compiled NumPy ufuncs, which behave like
built-in ufuncs. Let’s consider a Python implementation of numpy.add:


In [100]:
from numba import vectorize
@vectorize
def nb_add(x, y):
    return x + y

ModuleNotFoundError: No module named 'numba'

Now we have:


In [101]:
x = np.arange(10)
nb_add(x, x)

NameError: name 'nb_add' is not defined

In [102]:
nb_add.accumulate(x, 0)

NameError: name 'nb_add' is not defined

## A.8 Advanced Array Input and Output

In Chapter 4, we became acquainted with np.save and np.load for storing arrays in
binary format on disk. There are a number of additional options to consider for more
sophisticated use. In particular, memory maps have the additional benefit of enabling
you to work with datasets that do not fit into RAM.

## Memory-Mapped Files

A memory-mapped file is a method for interacting with binary data on disk as though
it is stored in an in-memory array. NumPy implements a memmap object that is
ndarray-like, enabling small segments of a large file to be read and written without
reading the whole array into memory. Additionally, a memmap has the same methods
as an in-memory array and thus can be substituted into many algorithms where an
ndarray would be expected.

To create a new memory map, use the function np.memmap and pass a file path, dtype,
shape, and file mode:

In [103]:
mmap = np.memmap('mymmap', dtype='float64', mode='w+',shape=(10000, 10000))
mmap

memmap([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

Slicing a memmap returns views on the data on disk:

In [104]:
section = mmap[:5]

If you assign data to these, it will be buffered in memory (like a Python file object),
but you can write it to disk by calling flush:


In [105]:
section[:] = np.random.randn(5, 10000)
mmap.flush()
mmap

memmap([[-0.18245763, -0.98387605, -0.03924531, ..., -2.1232373 ,
          0.4699011 ,  1.23045698],
        [ 0.27941134,  0.30688684, -0.62014901, ..., -1.99251109,
          0.03710872,  0.02795918],
        [ 1.27323018,  0.25729472,  1.17426959, ..., -0.57735952,
          0.89564726, -1.02518372],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

Whenever a memory map falls out of scope and is garbage-collected, any changes will
be flushed to disk also. When opening an existing memory map, you still have to spec‐
ify the dtype and shape, as the file is only a block of binary data with no metadata on
disk:

In [106]:
mmap = np.memmap('mymmap', dtype='float64', shape=(10000, 10000))
mmap

memmap([[-0.18245763, -0.98387605, -0.03924531, ..., -2.1232373 ,
          0.4699011 ,  1.23045698],
        [ 0.27941134,  0.30688684, -0.62014901, ..., -1.99251109,
          0.03710872,  0.02795918],
        [ 1.27323018,  0.25729472,  1.17426959, ..., -0.57735952,
          0.89564726, -1.02518372],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

Memory maps also work with structured or nested dtypes as described in a previous
section.

## HDF5 and Other Array Storage Options


PyTables and h5py are two Python projects providing NumPy-friendly interfaces for
storing array data in the efficient and compressible HDF5 format (HDF stands for
hierarchical data format). You can safely store hundreds of gigabytes or even terabytes
of data in HDF5 format. To learn more about using HDF5 with Python, I recommend
reading the pandas online documentation.


## A.9 Performance Tips

Getting good performance out of code utilizing NumPy is often straightforward, as
array operations typically replace otherwise comparatively extremely slow pure
Python loops. The following list briefly summarizes some things to keep in mind:
 
    
• Convert Python loops and conditional logic to array operations and boolean
array operations
  
    
• Use broadcasting whenever possible
  
   
• Use arrays views (slicing) to avoid copying data
   
    
• Utilize ufuncs and ufunc methods


If you can’t get the performance you require after exhausting the capabilities provided
by NumPy alone, consider writing code in C, Fortran, or Cython. I use Cython fre‐
quently in my own work as an easy way to get C-like performance with minimal
development.


## The Importance of Contiguous Memory

While the full extent of this topic is a bit outside the scope of this book, in some
applications the memory layout of an array can significantly affect the speed of com‐
putations. This is based partly on performance differences having to do with the
cache hierarchy of the CPU; operations accessing contiguous blocks of memory (e.g.,
summing the rows of a C order array) will generally be the fastest because the mem‐
ory subsystem will buffer the appropriate blocks of memory into the ultrafast L1 or
L2 CPU cache. Also, certain code paths inside NumPy’s C codebase have been opti‐
mized for the contiguous case in which generic strided memory access can be
avoided.


To say that an array’s memory layout is contiguous means that the elements are stored
in memory in the order that they appear in the array with respect to Fortran (column
major) or C (row major) ordering. By default, NumPy arrays are created as Ccontiguous or just simply contiguous. A column major array, such as the transpose of
a C-contiguous array, is thus said to be Fortran-contiguous. These properties can be
explicitly checked via the flags attribute on the ndarray:


In [107]:
arr_c = np.ones((1000, 1000), order='C')
arr_f = np.ones((1000, 1000), order='F')
arr_c.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [108]:
arr_f.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [109]:
arr_f.flags.f_contiguous

True

In this example, summing the rows of these arrays should, in theory, be faster for
arr_c than arr_f since the rows are contiguous in memory. Here I check for sure
using %timeit in IPython:

In [110]:
%timeit arr_c.sum(1)

2.43 ms ± 126 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [111]:
%timeit arr_f.sum(1)

885 µs ± 141 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


When you’re looking to squeeze more performance out of NumPy, this is often a
place to invest some effort. If you have an array that does not have the desired mem‐
ory order, you can use copy and pass either 'C' or 'F':

In [112]:
arr_f.copy('C').flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

When constructing a view on an array, keep in mind that the result is not guaranteed
to be contiguous:

In [113]:
arr_c[:50].flags.contiguous

True

In [114]:
arr_c[:, :50].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False